In [ ]:
#https://github.com/devam2905/MNIST-siamese-network/blob/master/MNIST_siamese_network.ipynb

In [2]:
import keras
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout,Flatten, Lambda
from keras.layers import Conv2D, Activation,AveragePooling2D,MaxPooling2D
from keras.optimizers import RMSprop
from keras import backend as K

import os
from pathlib import Path
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2

2024-05-13 10:33:53.646674: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 10:33:53.646723: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 10:33:53.646768: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-13 10:33:53.656842: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
full_ds = keras.utils.image_dataset_from_directory(
    directory="../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset",
    labels='inferred',
    label_mode='categorical',
    #color_mode='grayscale',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )

Found 1205260 files belonging to 73 classes.


In [43]:
basepath = Path('../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/')
files_in_basepath = basepath.iterdir()

for item in files_in_basepath:
    print(item)

../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/zoea__Decapoda
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/Oikopleuridae
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/ephyra
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/Oncaeidae
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/Euchaetidae
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/Salpida
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/nauplii__Cirripedia
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/Harpacticoida
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/Echinoidea
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/Eucalanidae
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/Obelia
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/detritus
../../../GlobalZooScanImageDataset/GlobalZooScanImageDataset/Brachyura
../../../GlobalZooScanImageDataset/GlobalZooScanImageDatas

In [3]:
data = []
label = []
path= ["../../dataset/global/test"]
basepath = Path('../../dataset/global/test')
files_in_basepath = basepath.iterdir()


for files in files_in_basepath:
    cpath = files
    print(files.name)
    #cpath = os.path.join(cpath, 'images')
    for img in os.listdir(cpath):
        image_array = cv2.imread(os.path.join(cpath, img), cv2.IMREAD_COLOR)
        data.append(image_array)
        label.append(files.name)


zoea__Decapoda
Oikopleuridae
ephyra
Oncaeidae
Euchaetidae
Salpida
nauplii__Cirripedia
Harpacticoida
Echinoidea
Eucalanidae
Obelia
detritus
Brachyura
Oithonidae
Acartiidae
Calanidae
Podon
Rhincalanidae
Evadne
Calanoida
Centropagidae
metanauplii
Chaetognatha
Corycaeidae
Fritillariidae
cyphonaute
Actinopterygii
Calocalanus pavo
calyptopsis
Ophiuroidea
Pyrosomatida
nauplii__Crustacea
Copilia
megalopa
Coscinodiscus
Spumellaria
Cavoliniidae
Euchirella
Luciferidae
Aglaura
zoea__Galatheidae
Limacinidae
cypris
Rhopalonema
Scyphozoa
Pontellidae
Sapphirinidae
Limacidae
larvae__Crustacea
Doliolida
larvae__Annelida
Hyperiidea
Foraminifera
Ostracoda
Bivalvia__Mollusca
Phaeodaria
cirrus
Annelida
Haloptilus
Creseidae
Noctiluca
Penilia
Neoceratium
Acantharea
Tomopteridae
Decapoda
Candaciidae
Temoridae
Gammaridea
Ctenophora


In [4]:
data = np.asarray(data, dtype="object")
label = np.asarray(label)

In [5]:
data

array([array([[[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              ...,

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
               [255, 255, 255],
               [255, 255, 255],
               [255, 255, 255]],

              [[255, 255, 255],
               [255, 255, 255],
               [255, 255, 255],
               ...,
        

In [37]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="../../dataset/global_10/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=730,
    #color_mode='grayscale',
    image_size=(224, 224)
    )
val_ds = keras.utils.image_dataset_from_directory(
    directory="../../dataset/global_10/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=730,
    #color_mode='grayscale',
    image_size=(224, 224)
    )

Found 730 files belonging to 73 classes.


2024-05-12 21:38:04.263105: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31125 MB memory:  -> device: 0, name: Tesla V100-SXM3-32GB, pci bus id: 0000:5c:00.0, compute capability: 7.0


Found 730 files belonging to 73 classes.


In [46]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1. / 255, 
    shear_range = 0.2, 
    zoom_range = 0.2, 
    horizontal_flip = True,
    vertical_flip = True,
    rotation_range = 180,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    validation_split = 0.2) 

train_generator = data_generator.flow_from_directory(
    "../../dataset/global_10/train", 
    target_size =(224, 224), 
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical',
    seed = 42,
    subset='training')

validation_generator = data_generator.flow_from_directory( 
    "../../dataset/global_10/train", 
    target_size =(224, 224), 
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical',
    seed = 42,
    subset='validation') 




Found 584 images belonging to 73 classes.
Found 146 images belonging to 73 classes.


In [3]:
for x_train, y_train in train_ds.take(1):
  print(x_train.shape, y_train.shape)


for x_test, y_test in val_ds.take(1):
  print(x_test.shape, y_test.shape)

(730, 224, 224, 1) (730, 73)
(730, 224, 224, 1) (730, 73)


In [47]:
num_classes=73
epochs=20

In [21]:
num_classes

73

In [48]:
def euclid_dis(vects):
  x,y = vects
  sum_square = K.sum(K.square(x-y), axis=1, keepdims=True)
  return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    y_true = float(y_true)
    y_pred = float(y_pred)
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

In [49]:
def create_pairs(x, digit_indices):
  pairs = []
  labels = []
  
  n=min([len(digit_indices[d]) for d in range(num_classes)]) -1
  
  for d in range(num_classes):
    for i in range(10):
      z1, z2 = digit_indices[d][i], digit_indices[d][i+1]
      pairs += [[x[z1], x[z2]]]
      inc = random.randrange(1, num_classes)
      dn = (d + inc) % num_classes
      z1, z2 = digit_indices[d][i], digit_indices[dn][i]
      pairs += [[x[z1], x[z2]]]
      labels += [1,0]

    #print(labels)
  return np.array(pairs), np.array(labels)

In [50]:
def create_base_net(input_shape):
  
  input = Input(shape = input_shape)
  x = Conv2D(32, (3,3), activation = 'relu')(input)
  x = AveragePooling2D(pool_size = (2,2))(x)
  x = Conv2D(64, (3,3), activation = 'tanh')(x)
  x = MaxPooling2D(pool_size = (2,2))(x)
  x = Dropout(0.25)(x)
  x = Flatten()(x)
  x = Dense(128, activation = 'tanh')(x)#128
  x = Dropout(0.5)(x)
  x = Dense(64,activation = 'tanh')(x)
  x = Dropout(0.5)(x)
  x = Dense(10,activation = 'tanh')(x)
  model = Model(input, x)
  model.summary()
  
  return model

In [52]:

# the data, split between train and test sets
x_train, y_train = next(train_generator)
x_test, y_test = next(validation_generator)


x_train = x_train.reshape(x_train.shape[0], 224, 224,3)
x_test = x_test.reshape(x_test.shape[0], 224, 224, 3)
# input_shape = (1, 28, 28)
print(x_train.shape)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

input_shape = x_train.shape[1:]
#input_shape = (28, 28, 1)

(32, 224, 224, 3)


In [53]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [55]:
# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(x_test, digit_indices)

# network definition
base_network = create_base_net(input_shape)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 average_pooling2d (AverageP  (None, 111, 111, 32)     0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 54, 54, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 54, 54, 64)        0     

In [56]:
tr_pairs[:, 0]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [57]:
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclid_dis,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

In [58]:
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed